In [2]:
# Python3 program to find maximum
# achievable value with a knapsack
# of weight W and multiple instances allowed.

# Returns the maximum value
# with knapsack of W capacity
def unboundedKnapsack(W, n, val, wt):

    # dp[i] is going to store maximum
    # value with knapsack capacity i.
    dp = [0 for i in range(W + 1)]

    ans = 0

    # Fill dp[] using above recursive formula
    for i in range(W + 1):
        for j in range(n):
            if (wt[j] <= i):
                dp[i] = max(dp[i], dp[i - wt[j]] + val[j])

    return dp[W]

# Driver program
W = 100
val = [25, 24, 15, 30]
wt = [18, 15, 10, 15]
n = len(val)

print(unboundedKnapsack(W, n, val, wt))

# This code is contributed by Anant Agarwal.

195


In [42]:
import numpy as np

# Calculate W_prime
def calculate_W_prime(W, w1):
    return W - (W // w1) * w1

# Calculate z_prime
def calculate_z_prime(W, w1, v1, w2, v2):
    floor_div_w1 = W // w1
    W_prime = calculate_W_prime(W, w1)
    floor_div_w2 = W_prime // w2
    return floor_div_w1 * v1 + floor_div_w2 * v2

# Calculate W_double_prime
def calculate_W_double_prime(W, w1, w2):
    W_prime = calculate_W_prime(W, w1)
    return W_prime - (W_prime // w2) * w2

# Calculate U_prime
def calculate_U_prime(W, w1, v1, w2, v2, w3, v3):
    z_prime = calculate_z_prime(W, w1, v1, w2, v2)
    W_double_prime = calculate_W_double_prime(W, w1, w2)
    floor_div_w3 = W_double_prime // w3
    return z_prime + floor_div_w3 * v3

# Calculate U_double_prime
def calculate_U_double_prime(W, w1, v1, w2, v2, w3, v3):
    z_prime = calculate_z_prime(W, w1, v1, w2, v2)
    W_double_prime = calculate_W_double_prime(W, w1, w2)
    term1 = W_double_prime + (1 / w1) * (w2 - W_double_prime) * w1
    # floor_div_w2 = W_double_prime // w2
    floor_div_w1 = (w2 - W_double_prime) // w1
    return z_prime + np.floor(term1 * v2 / w2 - floor_div_w1 * v1)

# Calculate U
def calculate_U(W, w1, v1, w2, v2, w3, v3):
    U_prime = calculate_U_prime(W, w1, v1, w2, v2, w3, v3)
    U_double_prime = calculate_U_double_prime(W, w1, v1, w2, v2, w3, v3)
    return max(U_prime, U_double_prime)

# Procedure 1: Eliminate dominated items
def eliminate_dominated_items(N, v, w):
    indices_to_remove = set()
    for j in range(len(N) - 1):
        for k in range(j + 1, len(N)):
            if w[N[k]] >= w[N[j]] and v[N[k]] <= v[N[j]]:
                indices_to_remove.add(k)
            elif w[N[k]] <= w[N[j]] and v[N[k]] >= v[N[j]]:
                indices_to_remove.add(j)

    filtered_N = [idx for idx in range(len(N)) if idx not in indices_to_remove]
    return filtered_N


# Step 1: Initialize
def initialize(W, v, w):
    N = list(range(len(v)))
    # print(N)
    N = eliminate_dominated_items(N, v, w)
    N.sort(key=lambda i: v[i] / w[i], reverse=True)
    x_hat, x, i, z_hat = 0, np.zeros(len(N)), 0, 0
    M = np.empty((len(N), W + 1))  
    x1 = int(np.floor(W / w[N[0]]))
    V_N = v[N[0]] * x1
    W_prime = W - w[N[0]] * x1
    x[0] = x1
    U = calculate_U(W, w[N[0]], v[N[0]], w[N[min(1, len(N) - 1)]], v[N[min(1, len(N) - 1)]], w[N[min(2, len(N) - 1)]], v[N[min(2, len(N) - 1)]])  # Calculate U
    mi = [min(w[j+1:], default=None) for j in range(len(N))]
    # print(mi)
    # print("mi")
    print("step 1 finished","i:", i, "N:", N, "x:",x,"x_hat",x_hat,"z_hat:", z_hat, "V_N:", V_N, "W_prime:", W_prime, "mi:", mi, "U:", U, "v:", v, "w:", w)
    return N, x_hat, x, i, z_hat, M, V_N, W_prime, U, mi


# Step 2
def step_2(W_prime, mi, N, V_N, U, z_hat, x, x_hat, M, v, w, i):
    print("step 2 start","i:", i, "N:", N, "x:",x,"x_hat",x_hat,"z_hat:", z_hat, "V_N:", V_N, "W_prime:", W_prime, "mi:", mi, "U:", U, "v:", v, "w:", w)

    if W_prime < mi[i]:
        print("in W_prime < mi[i] cond")
        if z_hat < V_N:
            z_hat = V_N
            x_hat = x.copy()
            if z_hat == U:
                return z_hat, x_hat
        return step_3(N, x_hat, x, z_hat, V_N, W_prime, mi, U, M, v, w, i)
    else:
        print("in else cond")
        j = next((j for j in range(i + 1, len(N)) if w[N[j]] <= W_prime), None)
        print("j: ", j)
        if j is None:
            return step_3(N, x_hat, x, z_hat, V_N, W_prime, mi, U, M, v, w, i - 1)
        temp = calculate_U(W_prime, w[N[i]], v[N[i]], w[N[min(i + 1, len(N) - 1)]], v[N[min(i + 1, len(N) - 1)]], w[N[min(i + 2, len(N) - 1)]], v[N[min(i + 2, len(N) - 1)]])
        if V_N + temp <= z_hat:
            return step_3(N, x_hat, x, z_hat, V_N, W_prime, mi, U, M, v, w, i)

        if mi[j] >= V_N:
            return step_3(N, x_hat, x, z_hat, V_N, W_prime, mi, U, M, v, w, i)

        x_j = int(np.floor(W_prime / w[N[j]]))
        
        V_N += v[N[j]] * x_j
        W_prime -= w[N[j]] * x_j
        M[i, W_prime] = V_N
        x[j] = x_j
        i = j
        return step_2(W_prime, mi, N, V_N, U, z_hat, x, x_hat, M, v, w, i)




# Step 3: Backtrack
def step_3(N, x_hat, x, z_hat, V_N, W_prime, mi, U, M, v, w, i):
    print("step 3 start","i:", i, "N:", N,"x:",x,"x_hat",x_hat, "z_hat:", z_hat, "V_N:", V_N, "W_prime:", W_prime, "mi:", mi, "U:", U, "v:", v, "w:", w)
    j = max([j for j in range(i + 1) if x[j] > 0], default=-1)
    print('j: ', j)
    if j < 0:
        return step_5(z_hat, x_hat)
    i = j
    x[i] -= 1
    V_N -= v[N[i]]
    W_prime += w[N[i]]
    if W_prime < mi[i]:
        return step_3(N, x_hat, x, z_hat, V_N, W_prime, mi, U, M, v, w, i)
    if i+1 < len(N):
        if V_N + int(np.floor(W_prime * (v[N[i+1]]) / (w[N[i+1]]))) <= z_hat:
            V_N -= v[N[i]] * x[i]
            W_prime += w[N[i]] * x[i]
            x[i] = 0
            return step_3(N, x_hat, x, z_hat, V_N, W_prime, mi, U, M, v, w, i)
    if W_prime >= mi[i]:
        return step_2(W_prime, mi, N, V_N, U, z_hat, x, x_hat, M, v, w, i)

# Step 4: Replace a jth item with an hth item
def step_4(i, N, x, x_hat, z_hat, V_N, W_prime, mi, U, M, v, w):
    print("step 4 start","i:", i, "N:", N,  "x:",x,"x_hat",x_hat,"z_hat:", z_hat, "V_N:", V_N, "W_prime:", W_prime, "mi:", mi, "U:", U, "v:", v, "w:", w)
    j, h = i, i + 1
    if z_hat >= V_N + int(np.floor(W_prime * v[N[h]] / w[N[h]])):
        return step_3(N, x_hat, x, z_hat, V_N, W_prime, mi, U, M, v, w, i)
    if w[N[h]] >= w[N[j]]:
        if w[N[h]] == w[N[j]] or w[N[h]] > W_prime or z_hat >= V_N + v[N[h]]:
            h += 1
            return step_4(i, N, x, x_hat, z_hat, V_N, W_prime, mi, U, M, v, w)
        z_hat = V_N + v[N[h]]
        x_hat = x.copy()
        x_hat[h] = 1
        if z_hat == U:
            return step_5(z_hat, x_hat)
        j = h
        h += 1
        return step_4(i, N, x, x_hat, z_hat, V_N, W_prime, mi, U, M, v, w)
    else:
        if W_prime - w[N[h]] < mi[h - 1]:
            h += 1
            return step_4(i, N, x, x_hat, z_hat, V_N, W_prime, mi, U, M, v, w)
        i = h
        x[i] = int(np.floor(W_prime / w[N[i]]))
        V_N += v[N[i]] * x[i]
        W_prime -= w[N[i]] * x[i]
        return step_2(W_prime, mi, N, V_N, U, z_hat, x, x_hat, M, v, w, i)

# Step 5: Finish
def step_5(z_hat, x_hat):
    return z_hat, x_hat

# Example usage
W_val = 100
v_values = [25, 24, 15, 30]
w_values = [18, 15, 10, 15]

N, x_hat, x, i, z_hat, M, V_N, W_prime, U, mi = initialize(W_val, v_values, w_values)
result = step_2(W_prime, mi, N, V_N, U, z_hat, x, x_hat, M, v_values, w_values, i)
print("Optimal value:", result[0])
print("Optimal steps:", result[1])

step 1 finished i: 0 N: [3, 2] x: [6. 0.] x_hat 0 z_hat: 0 V_N: 180 W_prime: 10 mi: [10, 10] U: 210.0 v: [25, 24, 15, 30] w: [18, 15, 10, 15]
step 2 start i: 0 N: [3, 2] x: [6. 0.] x_hat 0 z_hat: 0 V_N: 180 W_prime: 10 mi: [10, 10] U: 210.0 v: [25, 24, 15, 30] w: [18, 15, 10, 15]
in else cond
j:  1
step 2 start i: 1 N: [3, 2] x: [6. 1.] x_hat 0 z_hat: 0 V_N: 195 W_prime: 0 mi: [10, 10] U: 210.0 v: [25, 24, 15, 30] w: [18, 15, 10, 15]
in W_prime < mi[i] cond
step 3 start i: 1 N: [3, 2] x: [6. 1.] x_hat [6. 1.] z_hat: 195 V_N: 195 W_prime: 0 mi: [10, 10] U: 210.0 v: [25, 24, 15, 30] w: [18, 15, 10, 15]
j:  1
step 2 start i: 1 N: [3, 2] x: [6. 0.] x_hat [6. 1.] z_hat: 195 V_N: 180 W_prime: 10 mi: [10, 10] U: 210.0 v: [25, 24, 15, 30] w: [18, 15, 10, 15]
in else cond
j:  None
step 3 start i: 0 N: [3, 2] x: [6. 0.] x_hat [6. 1.] z_hat: 195 V_N: 180 W_prime: 10 mi: [10, 10] U: 210.0 v: [25, 24, 15, 30] w: [18, 15, 10, 15]
j:  0
step 3 start i: 0 N: [3, 2] x: [0. 0.] x_hat [6. 1.] z_hat: 195 

In [11]:
import os
import pickle
import random
import time
import psutil

# Define your unboundedKnapsack and other necessary functions
# (Assuming these functions are defined elsewhere in your code)

# Function to generate or load dataset
def generate_or_load_dataset(num_items, path):
    if os.path.exists(path):
        with open(path, 'rb') as file:
            return pickle.load(file)
    else:
        val = [random.randint(10, 100) for _ in range(num_items)]
        wt = [random.randint(20, 200) for _ in range(num_items)]
        W = num_items
        dataset = (W, val, wt)
        with open(path, 'wb') as file:
            pickle.dump(dataset, file)
        return dataset

def test_algorithms(num_items, path):
    W, val, wt = generate_or_load_dataset(num_items, path)

    start_time = time.time()
    psutil.Process(os.getpid()).memory_info().rss
    memory_before = psutil.virtual_memory().used
    # Execute the first algorithm
    n = len(val)
    result_1 = unboundedKnapsack(W, n, val, wt)
    first_algorithm_time = time.time() - start_time
    memory_after = psutil.virtual_memory().used
    first_algorithm_memory = memory_after - memory_before

    # Use the same dataset for the second algorithm
    W_val = W
    v_values = val
    w_values = wt

    start_time = time.time()
     
    memory_before = psutil.virtual_memory().used
    # Execute the second algorithm
    N, x_hat, x, i, z_hat, M, V_N, W_prime, U, mi = initialize(W_val, v_values, w_values)
    result_2 = step_2(W_prime, mi, N, V_N, U, z_hat, x, x_hat, M, v_values, w_values, i)
    result_2 = result_2[0]
    second_algorithm_time = time.time() - start_time
    memory_after = psutil.virtual_memory().used
    second_algorithm_memory = memory_after - memory_before

    print(f"Results for Algorithm 1: {result_1}")
    print(f"Results for Algorithm 2: {result_2}")

    return first_algorithm_time, second_algorithm_time, first_algorithm_memory,second_algorithm_memory

# List of dataset sizes to be tested
num_items_list = [100, 1000, 10000]  # Smaller dataset sizes

for num_items in num_items_list:
    psutil.Process(os.getpid()).memory_info().rss
    path = f"dataset_{num_items}.pickle"
    time_1, time_2,memo_1, memo_2 = test_algorithms(num_items, path)
    print(f"Number of items: {num_items}")
    print(f"Execution time for Algorithm 1: {time_1} seconds")
    print(f"Execution time for Algorithm 2: {time_2} seconds")
    print(f"Memory used for Algorithm 1: {memo_1} ")
    print(f"Memory used for Algorithm 2: {memo_2} ")



Results for Algorithm 1: 249
Results for Algorithm 2: 249
Number of items: 100
Execution time for Algorithm 1: 0.0009982585906982422 seconds
Execution time for Algorithm 2: 0.0199961662292480472 seconds
Memory used for Algorithm 1: 240
Memory used for Algorithm 2: 20480 
Results for Algorithm 1: 4850
Results for Algorithm 2: 4850
Number of items: 1000
Execution time for Algorithm 1: 0.2690696716308594 seconds
Execution time for Algorithm 2: 0.11451578140258789 seconds
Memory used for Algorithm 1: 294912 
Memory used for Algorithm 2: 5496832 
Results for Algorithm 1: 50000
Results for Algorithm 2: 50000
Number of items: 10000
Execution time for Algorithm 1: 29.704813718795776 seconds
Execution time for Algorithm 2: 1.2950119972229 seconds
Memory used for Algorithm 1: 14434304 
Memory used for Algorithm 2: 78912132


